In [1]:
import torch

# Check if CUDA is available
print("CUDA available:", torch.cuda.is_available())

# If True, print the name of the GPU
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


CUDA available: True
GPU: NVIDIA GeForce GTX 1050 Ti


In [2]:
import math
import random
import numpy as np
import struct
import sys

In [3]:
sys.setrecursionlimit(10000)

In [4]:
class Value:
  
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"
  
  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')
    
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    
    return out

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')
    
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
      
    return out
  
  def __pow__(self, other):
    assert isinstance(other, (int, float))
    out = Value(self.data**other, (self,), f'**{other}')

    def _backward():
        self.grad += other * (self.data ** (other - 1)) * out.grad
    out._backward = _backward

    return out
  
  def __rmul__(self, other): # other * self
    return self * other

  def __truediv__(self, other): # self / other
    return self * other**-1

  def __neg__(self): # -self
    return self * -1

  def __sub__(self, other): # self - other
    return self + (-other)

  def __radd__(self, other): # other + self
    return self + other

  def tanh(self):
    x = self.data
    t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
    out = Value(t, (self, ), 'tanh')
    
    def _backward():
      self.grad += (1 - t**2) * out.grad
    out._backward = _backward
    
    return out
  
  def exp(self):
    x = self.data
    out = Value(math.exp(x), (self, ), 'exp')
    
    def _backward():
      self.grad += out.data * out.grad
    out._backward = _backward
    
    return out
  
  
  def backward(self):
    
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    
    self.grad = 1.0
    for node in reversed(topo):
      node._backward()



In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random

# Neuron equivalent in PyTorch
class Neuron(nn.Module):
    def __init__(self, nin):
        super(Neuron, self).__init__()
        self.fc = nn.Linear(nin, 1)  # Fully connected layer for a single output
  
    def forward(self, x):
        act = self.fc(x)  # Linear transformation
        out = torch.tanh(act)  # Apply Tanh activation function
        return out

# Layer equivalent in PyTorch
class Layer(nn.Module):
    def __init__(self, nin, nout):
        super(Layer, self).__init__()
        self.neurons = nn.ModuleList([Neuron(nin) for _ in range(nout)])  # Create `nout` neurons
  
    def forward(self, x):
        outs = [neuron(x) for neuron in self.neurons]  # Forward pass through each neuron
        return outs[0] if len(outs) == 1 else torch.cat(outs, dim=-1)  # Return single output or concatenate outputs

# MLP (Multi-Layer Perceptron) equivalent in PyTorch
class MLP(nn.Module):
    def __init__(self, nin, nouts):
        super(MLP, self).__init__()
        sz = [nin] + nouts
        self.layers = nn.ModuleList([Layer(sz[i], sz[i+1]) for i in range(len(nouts))])  # Create layers
  
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)  # Pass input through each layer
        return x

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n = MLP(784, [523, 10])
n =  n.to(device)

In [7]:
def read_idx3_ubyte(file_path):
    with open(file_path, 'rb') as f:
        # Read the magic number
        magic, num_images, rows, cols = struct.unpack(">IIII", f.read(16))
        
        # Read the image data
        images = np.fromfile(f, dtype=np.uint8).reshape(num_images, rows, cols)
        
    return images

def read_idx1_ubyte(file_path):
    with open(file_path, 'rb') as f:
        # Read the magic number and number of items
        magic, num_items = struct.unpack(">II", f.read(8))
        
        # Read the label data
        labels = np.fromfile(f, dtype=np.uint8)
        
    return labels

# Function to convert labels to one-hot encoding
def one_hot_encode(labels, num_classes=10):
    one_hot_labels = np.zeros((len(labels), num_classes))
    one_hot_labels[np.arange(len(labels)), labels] = 1
    return one_hot_labels

# Example usage:
image_file_path = 'C:\\Users\\Shadow\\Downloads\\MNIST_ORG\\t10k-images.idx3-ubyte'
label_file_path = 'C:\\Users\\Shadow\\Downloads\\MNIST_ORG\\t10k-labels.idx1-ubyte'

# Load images and labels
images = read_idx3_ubyte(image_file_path)
labels = read_idx1_ubyte(label_file_path)

# Limit to the first 1000 images and labels
limit = 1000
images = images[:limit]  # Get only the first 1000 images
labels = labels[:limit]  # Get only the first 1000 labels

print(f"Loaded {len(images)} images with size {images.shape[1]}x{images.shape[2]}")
print(f"Loaded {len(labels)} labels")

# Flatten the images (28x28 -> 784) and normalize
xs = [img.flatten() / 255.0 for img in images]
ys = one_hot_encode(labels)  

Loaded 1000 images with size 28x28
Loaded 1000 labels


In [8]:
def softmax(values):
    # Compute exponentials of all values
    exp_vals = [v.exp() for v in values]
    
    # Sum of exponentials
    sum_exp_vals = sum(exp_vals)
    
    # Normalize each value
    softmax_vals = [v / sum_exp_vals for v in exp_vals]
    return softmax_vals


In [ ]:
import torch
import numpy as np

# Check if CUDA is available and set the device accordingly
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Convert xs and ys to NumPy arrays before converting to tensors
xs_np = np.array(xs)  # Convert list of numpy arrays to a single numpy array
ys_np = np.array(ys)

# Now convert these NumPy arrays to PyTorch tensors
xs_tensor = torch.tensor(xs_np, dtype=torch.float32).to(device)
ys_tensor = torch.tensor(ys_np, dtype=torch.float32).to(device)

print('Starting Operation')
learning_rate = 0.5
for k in range(20):
    # Forward pass: compute predictions
    ypred = [n(x).to(device) for x in xs_tensor]

    # Ensure ypred is on the same device as ys_tensor
    ypred = [yp.to(device) for yp in ypred]
    
    # Your softmax and loss operations
    ypred_softmax = [softmax(yp) for yp in ypred]

    # Loss calculation, ensuring both predicted and true values are on GPU
    loss = torch.tensor(0., device=device)
    for i in range(len(ys_tensor)):
        ytrue = ys_tensor[i]
        for j in range(len(ytrue)):
            y_pred_prob = ypred_softmax[i][j].to(device)
            y_true_val = ytrue[j].to(device)
            loss += ((y_pred_prob - y_true_val)**2).to(device)

    loss /= len(ys_tensor)
    print(f"Iteration {k}, Loss: {loss.item()}")

    # Backward pass
    for p in n.parameters():
        p.grad = None

    loss.backward()

   # if k > 7:
    #    learning_rate = max(0.1, learning_rate - 0.5)

    # Update parameters
    with torch.no_grad():
        for p in n.parameters():
            p.data.add_(-learning_rate * p.grad)


Starting Operation
Iteration 0, Loss: 0.9019241333007812


In [9]:
import torch
from PIL import Image
import numpy as np


def image_to_binary(image_path, threshold=128):
    
    img = Image.open(image_path).convert('L')  
    
    
    img_np = np.array(img)
    
    
    binary_img = (img_np > threshold).astype(np.uint8)
    
    return binary_img


def prepare_tensor_for_model(binary_img):
    
    binary_tensor = torch.tensor(binary_img, dtype=torch.float32).unsqueeze(0)
    return binary_tensor

# Example usage
image_path = 'C:\\Users\\Shadow\\Documents\\nn_number_test\\3.png'
binary_image = image_to_binary(image_path)
binary_tensor = prepare_tensor_for_model(binary_image)


binary_tensor = binary_tensor.view(-1, 28*28).to(device)


output = n(binary_tensor)
print(output)


tensor([[ 0.0542,  0.0652, -0.0181,  0.0588, -0.1418,  0.1642, -0.2404,  0.0088,
          0.1769,  0.0257]], device='cuda:0', grad_fn=<CatBackward0>)
